# Get tweets from Twitter API

In [20]:
## Code from: https://stackabuse.com/accessing-the-twitter-api-with-python/#disqus_thread

# Import the Twython class
from twython import Twython
import json

import pandas as pd

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Number of users
Number_of_users = 50


## QUERIES!
## Create our queries

query1 = {'q': 'climate',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query2 = {'q': 'businesses',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query3 = {'q': 'politics',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query4 = {'q': 'economy',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query5 = {'q': '#innovation',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query6 = {'q': 'nature',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query7 = {'q': 'culture',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query8 = {'q': 'healthcare',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query9 = {'q': 'pop culture',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query10 = {'q': 'culture',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }
query11 = {'q': 'democrats',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

query12 = {'q': 'technology',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        'tweet_mode' : 'extended',
        }

# Search tweets
def search(query):
    dict_ = {'user': [], 'date': [], 'hashtags': [], 'full_text': [], 'favorite_count': []}
    for status in python_tweets.search(**query)['statuses']:
        dict_['user'].append(status['user']['screen_name'])
        dict_['date'].append(status['created_at'])
        dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
        dict_['full_text'].append(status['full_text'])
        dict_['favorite_count'].append(status['favorite_count'])
    return dict_

# Note that these are the most popular tweets containing the word "..." in the past 7 days. 


df_q1 = pd.DataFrame(search(query1))
df_q2 = pd.DataFrame(search(query2))
df_q3 = pd.DataFrame(search(query3))
df_q4 = pd.DataFrame(search(query4))
df_q5 = pd.DataFrame(search(query5))
df_q6 = pd.DataFrame(search(query6))
df_q7 = pd.DataFrame(search(query7))
df_q8 = pd.DataFrame(search(query8))
df_q9 = pd.DataFrame(search(query9))
df_q10 = pd.DataFrame(search(query10))
df_q11 = pd.DataFrame(search(query11))
df_q12 = pd.DataFrame(search(query12))


df_total = pd.concat([df_q1,df_q2,df_q3,df_q4,df_q5,df_q6,df_q7,df_q8,df_q9,df_q10,df_q11,df_q12])

# Train model to identify topics

In [21]:
total_train_DF = pd.read_csv (r'train_data3.csv')
start_train_DF = total_train_DF.drop(columns=['char_count','word_count','word_density','punctuation_count','title_word_count',
                                              'upper_case_word_count','noun_count','verb_count','adj_count','adv_count',
                                              'pron_count'])
start_train_DF

## Code from: 
## https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/


from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(start_train_DF['full_text'], start_train_DF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

## labels that are used
topics = list(encoder.classes_)
topics

# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(start_train_DF['full_text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(start_train_DF['full_text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(start_train_DF['full_text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(start_train_DF['full_text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/glove.6B.300d.txt', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(start_train_DF['full_text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
start_train_DF['char_count'] = start_train_DF['full_text'].apply(len)
start_train_DF['word_count'] = start_train_DF['full_text'].apply(lambda x: len(x.split()))
start_train_DF['word_density'] = start_train_DF['char_count'] / (total_train_DF['word_count']+1)
start_train_DF['punctuation_count'] = start_train_DF['full_text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
start_train_DF['title_word_count'] = start_train_DF['full_text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
start_train_DF['upper_case_word_count'] = start_train_DF['full_text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

start_train_DF['noun_count'] = start_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'noun'))
start_train_DF['verb_count'] = start_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'verb'))
start_train_DF['adj_count'] = start_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'adj'))
start_train_DF['adv_count'] = start_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'adv'))
start_train_DF['pron_count'] = start_train_DF['full_text'].apply(lambda x: check_pos_tag(x, 'pron'))


# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:501: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [22]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [23]:
# ### Train models

# # Naive Bayes on Count Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
# print ("NB, Count Vectors: ", accuracy)

# # Naive Bayes on Word Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
# print ("NB, WordLevel TF-IDF: ", accuracy)

# # Naive Bayes on Ngram Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("NB, N-Gram Vectors: ", accuracy)

# # Naive Bayes on Character Level TF IDF Vectors
# accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
# print ("NB, CharLevel Vectors: ", accuracy)

# # Linear Classifier on Count Vectors
# accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
# print ("LR, Count Vectors: ", accuracy)

# # Linear Classifier on Word Level TF IDF Vectors
# accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
# print ("LR, WordLevel TF-IDF: ", accuracy)

# # Linear Classifier on Ngram Level TF IDF Vectors
# accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("LR, N-Gram Vectors: ", accuracy)

# # Linear Classifier on Character Level TF IDF Vectors
# accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
# print ("LR, CharLevel Vectors: ", accuracy)

# # SVM on Ngram Level TF IDF Vectors
# accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
# print ("SVM, N-Gram Vectors: ", accuracy)

# # RF on Count Vectors
# accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
# print ("RF, Count Vectors: ", accuracy)

# # RF on Word Level TF IDF Vectors
# accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
# print ("RF, WordLevel TF-IDF: ", accuracy)

# # Extereme Gradient Boosting on Count Vectors
# accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
# print ("Xgb, Count Vectors: ", accuracy)

# # Extereme Gradient Boosting on Word Level TF IDF Vectors
# accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
# print ("Xgb, WordLevel TF-IDF: ", accuracy)

# # Extereme Gradient Boosting on Character Level TF IDF Vectors
# accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
# print ("Xgb, CharLevel Vectors: ", accuracy)



# ## NB, Count Vectors:  0.7766666666666666
# ## NB, WordLevel TF-IDF:  0.78
# ## NB, N-Gram Vectors:  0.6233333333333333
# ## NB, CharLevel Vectors:  0.7533333333333333
# ## LR, Count Vectors:  0.8
# ## LR, WordLevel TF-IDF:  0.8033333333333333
# ## LR, N-Gram Vectors:  0.6266666666666667
# ## LR, CharLevel Vectors:  0.8333333333333334
# ## SVM, N-Gram Vectors:  0.5966666666666667
# ## RF, Count Vectors:  0.7966666666666666
# ## RF, WordLevel TF-IDF:  0.8033333333333333
# ## [16:12:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
# ## C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
# ##   warnings.warn(label_encoder_deprecation_msg, UserWarning)
# ## Xgb, Count Vectors:  0.77
# ## [16:12:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
# ## Xgb, WordLevel TF-IDF:  0.77
# ## [16:12:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
# ## Xgb, CharLevel Vectors:  0.8266666666666667

In [ ]:
####### To save the AI model


# import pickle

# model = ensemble.RandomForestClassifier()
# model.fit(xtrain_tfidf, train_y)
# # save the model to disk
# filename = 'AImodel2.sav'
# pickle.dump(model, open(filename, 'wb'))

In [24]:
import pickle

filename = 'AImodel2.sav'
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [25]:
def topic_labelling(df):
    ## Prepare data for input in the model
    test_tfidf =  tfidf_vect.transform(df['full_text'])

    ## Load data in model
    result = loaded_model.predict(test_tfidf)

    ## Get results
    Final_result = []

    for i in range(len(result)):
        Final_result.append(topics[result[i]]) 

    # Final_result
    df['topic'] = Final_result
    
    return df

In [26]:
topic_labelling(df_q1)
topic_labelling(df_q2)
topic_labelling(df_q3)
topic_labelling(df_q4)
topic_labelling(df_q5)
topic_labelling(df_q6)
topic_labelling(df_q7)
topic_labelling(df_q8)
topic_labelling(df_q9)
topic_labelling(df_q10)
topic_labelling(df_q11)
topic_labelling(df_q12)
topic_labelling(df_total)

,user,date,hashtags,full_text,favorite_count,topic
0,DonaldJTrumpJr,Wed Jun 09 21:39:33 +0000 2021,[],I guess now that the entire world knows his so...,15877,climate
1,chrislhayes,Wed Jun 09 18:30:53 +0000 2021,[],"An ""infrastructure bill"" that isn't climate-fo...",38390,climate
2,LiZaOutlives,Thu Jun 10 12:33:48 +0000 2021,[],Liza Minnelli has outlived the Keystone pipeli...,17491,climate
3,Sherrill4Moxy,Thu Jun 10 19:11:58 +0000 2021,[],"RT @MrMichaelBurkes: Biden now says ""climate c...",0,climate
4,GlobalUnion3,Thu Jun 10 19:11:57 +0000 2021,[],RT @ExtinctionR: Venezuela 🇻🇪 is on the frontl...,0,climate
...,...,...,...,...,...,...
95,education_24x7,Thu Jun 10 19:10:51 +0000 2021,"[Tech, NewsFlash]",RT @techinjektion: #Tech #NewsFlash 06/09\nFac...,0,politics
96,RockymeOfficial,Thu Jun 10 19:10:50 +0000 2021,[],RT @OfficialJiasmin: Download in Qie app in Pl...,0,culture
97,lae_lay,Thu Jun 10 19:10:49 +0000 2021,[],After all these years and all this technology ...,0,tech
98,anuragdubey777,Thu Jun 10 19:10:47 +0000 2021,[cloud],RT @Omkar_Raii: Remote work infrastructure wit...,0,tech


# Create fake user profiles

In [27]:
df1 = df_q1.iloc[:20, :]
df2 = df_q1.iloc[20:40, :]
df3 = df_q1.iloc[40:60, :]
df4 = df_q1.iloc[60:80, :]
df5 = df_q1.iloc[80:, :]

df6 = df_q2.iloc[:20, :]
df7 = df_q2.iloc[20:40, :]
df8 = df_q2.iloc[40:60, :]
df9 = df_q2.iloc[60:80, :]
df10 = df_q2.iloc[80:, :]

df11 = df_q3.iloc[:20, :]
df12 = df_q3.iloc[20:40, :]
df13 = df_q3.iloc[40:60, :]
df14 = df_q3.iloc[60:80, :]
df15 = df_q3.iloc[80:, :]

df16 = df_q4.iloc[:20, :]
df17 = df_q4.iloc[20:40, :]
df18 = df_q4.iloc[40:60, :]
df19 = df_q4.iloc[60:80, :]
df20 = df_q4.iloc[80:, :]

df21 = df_q5.iloc[:20, :]
df22 = df_q5.iloc[20:40, :]
df23 = df_q5.iloc[40:60, :]
df24 = df_q5.iloc[60:80, :]
df25 = df_q5.iloc[80:, :]

df26 = df_q6.iloc[:20, :]
df27 = df_q6.iloc[20:40, :]
df28 = df_q6.iloc[40:60, :]
df29 = df_q6.iloc[60:80, :]
df30 = df_q6.iloc[80:, :]

df31 = df_q7.iloc[:20, :]
df32 = df_q7.iloc[20:40, :]
df33 = df_q7.iloc[40:60, :]
df34 = df_q7.iloc[60:80, :]
df35 = df_q7.iloc[80:, :]

df36 = df_q8.iloc[:20, :]
df37 = df_q8.iloc[20:40, :]
df38 = df_q8.iloc[40:60, :]
df39 = df_q8.iloc[60:80, :]
df40 = df_q8.iloc[80:, :]

df41 = df_q9.iloc[:20, :]
df42 = df_q9.iloc[20:40, :]
df43 = df_q9.iloc[40:60, :]
df44 = df_q9.iloc[60:80, :]
df45 = df_q9.iloc[80:, :]

df46 = df_q10.iloc[:20, :]
df47 = df_q10.iloc[20:40, :]
df48 = df_q10.iloc[40:60, :]
df49 = df_q10.iloc[60:80, :]
df50 = df_q10.iloc[80:, :]

df51 = df_q11.iloc[:20, :]
df52 = df_q11.iloc[20:40, :]
df53 = df_q11.iloc[40:60, :]
df54 = df_q11.iloc[60:80, :]
df55 = df_q11.iloc[80:, :]

df56 = df_q12.iloc[:20, :]
df57 = df_q12.iloc[20:40, :]
df58 = df_q12.iloc[40:60, :]
df59 = df_q12.iloc[60:80, :]
df60 = df_q12.iloc[80:, :]

In [28]:
import itertools 

numbers = []
for i in range(1,61):
    numbers.append(i)

combinations = list(itertools.combinations(numbers, 5)) 

df_combinations = []
x = 0

for i in range(Number_of_users):

    df_combinations.append(['df'+str(combinations[i+x][0]),
                            'df'+str(combinations[i+x][1]),
                            'df'+str(combinations[i+x][2]),
                            'df'+str(combinations[i+x][3]),
                            'df'+str(combinations[i+x][4])])
    
    name = 'user'+str(i)
    globals()[name] = pd.concat([globals()[df_combinations[i][0]],
                                 globals()[df_combinations[i][1]],
                                 globals()[df_combinations[i][2]],
                                 globals()[df_combinations[i][3]],
                                 globals()[df_combinations[i][4]]])
    x = x + 40000

print('number of user:', len(df_combinations))

number of user: 50


# TF-IDF analyse and Jaccard similarity

In [10]:
### TF-IDF code from: https://www.youtube.com/watch?v=UvsQPsrZTK4

import pandas as pd
import nltk
import string
import re     ## Using regex to remove punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

In [11]:
def Text_filter_tweets(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = ''.join([i for i in text if not i.isdigit()])
    
    ## Lose stop words and tokenize the text
    textfile = open('stopwords_extra.txt')
    stopwords_extra = []
    for line in textfile:
        row_data = line.strip("\n").split()
        for i, item in enumerate(row_data):
            try:
                row_data[i] = float(item)
            except ValueError:
                pass
            data_g = row_data[i].replace('[', '')
        stopwords_extra.append(data_g)
    stopwords_total = stopwords.union(set(stopwords_extra))
    
    BOW = word_tokenize(text)

    filtered_list = []

    for w in BOW:
        if w not in stopwords_total:
            filtered_list.append(w)
    
    lemmatizer = WordNetLemmatizer()
    
    Lemma = []
    for word in filtered_list:
        Lemma.append(lemmatizer.lemmatize(word, pos ="v"))
    
    return Lemma


def numofwords(Lemma, uniqueWords):
    numofwords = dict.fromkeys(uniqueWords, 0)
    for word in Lemma:
        numofwords[word] += 1
    return numofwords


def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict


def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict


def computeTFIDF(tfBagOfwords, idfs):
    tfidf = {}
    for word, val in tfBagOfwords.items():
        tfidf[word] = val * idfs[word]
    return tfidf


def DF_user_profile(tfidf_s):
    df = pd.DataFrame(tfidf_s)
    top = df.apply(lambda s: s.abs().nlargest(20).index.tolist(), axis=1)
    df_top = pd.DataFrame([top])
#     df_top.columns = ['article1', 'article2', 'article3', 'article4', 'article5']

#     df_1 = pd.DataFrame([df_top['article1']])

    unnested_lst = []
    for col in df_top.columns:
        unnested_lst.append(df_top[col].apply(pd.Series).stack())
    result = pd.concat(unnested_lst, axis=1, keys=df_top.columns)

    result.reset_index(inplace=True)
    result.drop('level_0', axis='columns', inplace=True)
    result.drop('level_1', axis='columns', inplace=True)

    return result


def tfidf_short(tfidf):
#     tfidf_short = dict(sorted(tfidf.items(), key=lambda item: item[1], reverse=True)[:20])
    tfidf_short = dict(sorted(tfidf.items(), key=lambda item: item[1], reverse=True))

    return tfidf_short


def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [12]:
def TFIDF_analysis(user):
    ## 1
    list_user = user['full_text'].values.tolist()

    ## 2
    Lemma_user = []

    for i in range(len(user)):
        Lemma_user.append(Text_filter_tweets(list_user[i]))

    ## 3
    uniqueWords_user = []
    lenght = []

    for i in range(len(Lemma_user)):
        lenght.append(len(Lemma_user[i]))
        for a in range(len(Lemma_user[i])):
            uniqueWords_user.append(Lemma_user[i][a])

    ## 4
    uniqueWords_user = set(uniqueWords_user)

    ## 5
    numofword_user = list(())
    for i in range(len(Lemma_user)):
        numofword_user.append(numofwords(Lemma_user[i], uniqueWords_user))
    numofword_user = tuple(numofword_user)
    # numofword_user

    ## 6
    tf_user = list(())
    for i in range(len(Lemma_user)):
        tf_user.append(computeTF(numofword_user[i], Lemma_user[i]))
    tf_user = tuple(tf_user)
    # tf_user

    ## 7
    idfs_user = computeIDF(numofword_user)
    # idfs_user

    ## 8
    tfidf_user = list(())
    for i in range(len(tf_user)):
        tfidf_user.append(computeTF(tf_user[i], idfs_user))
    tfidf_user = tuple(tfidf_user)
    # tfidf_user

    ## 9
    for i in range(len(tfidf_user)):
        user_profile_user = {**tfidf_user[i]}
    # user_profile_user

    ## 10
    DF_user_profile(tfidf_user)
    
    return user_profile_user, tfidf_user

In [13]:
def numb_simularity():
    numbers_similarity = []

    for i in range(Number_of_users):
        numbers_similarity.append(i)
        # numbers_similarity

    combinations_similarity = list(itertools.combinations(numbers_similarity, 2))
    
    return combinations_similarity

In [14]:
def simularity_calculation(topic):
    for i in range(len(df_combinations)):
        name = 'user'+str(i)
        profile = 'user_profile'+str(i)
        tfidf_profile = 'tfidf_user'+str(i)
        values_returned = TFIDF_analysis(globals()[name][globals()[name]['topic'] == topic])
        globals()[profile] = values_returned[0]
        globals()[tfidf_profile] = values_returned[1]
    
#     numbers_similarity = []

#     for i in range(Number_of_users):
#         numbers_similarity.append(i)
#     # numbers_similarity

#     combinations_similarity = list(itertools.combinations(numbers_similarity, 2))
#     # combinations_similarity
    
    combinations_similarity = numb_simularity()
    
    m_dict = {}

    for i in range(len(combinations_similarity)):
        c = combinations_similarity[i]
        profile_A = 'user_profile'+str(combinations_similarity[i][0])
        profile_B = 'user_profile'+str(combinations_similarity[i][1])
        c_dict = {c: [jaccard_similarity(globals()[profile_A], globals()[profile_B])]}
        m_dict = {**m_dict, **c_dict}
    #     m_dict[globals()['profile_B']].append(jaccard_similarity(globals()[profile_A], globals()[profile_B]))
    
    return m_dict

In [15]:
def materix(m_dict):
        
    df_d = pd.DataFrame(m_dict)
    df_m = df_d[0]

    for i in range(Number_of_users-1):
        df_m = df_m.append(df_d[i])

    df_m = df_m.reset_index()
    df_m = df_m.drop([0], axis=0)
    df_m = df_m.reset_index()
    del df_m['index']
    del df_m['level_0']
    
    return df_m

In [29]:
## if the code crashes here, than try another twitter query that is related to the topics

m_dict_politics = simularity_calculation('politics')
df_m_politics = materix(m_dict_politics)

m_dict_climate = simularity_calculation('climate')
df_m_climate = materix(m_dict_climate)

m_dict_technology = simularity_calculation('tech')
df_m_technology = materix(m_dict_technology)

m_dict_culture = simularity_calculation('culture')
df_m_culture = materix(m_dict_culture)

m_dict_health = simularity_calculation('health')
df_m_health = materix(m_dict_health)

m_dict_business = simularity_calculation('business')
df_m_business = materix(m_dict_business)

In [ ]:
df_m_politics

# Pre-calculations

In [30]:
def sorting(m_dict, User_id):
    User_sim_list = []
    
    combinations_similarity = numb_simularity()

    for i in range(len(m_dict)):
            if combinations_similarity[i][0] is User_id:
                    User_sim_list.append([combinations_similarity[i][1], m_dict[combinations_similarity[i]]])
            elif combinations_similarity[i][1] is User_id:
                    User_sim_list.append([combinations_similarity[i][0], m_dict[combinations_similarity[i]]])

    User_sim_list.sort(key=lambda x:x[1])
    
    return User_sim_list

In [31]:
def mapp(user_sim_lists):
    Map_right = []
    Map_left = []

    for i in range(len(user_sim_lists)):
        if (i % 2 == 0): 
            Map_right.append(user_sim_lists[i])
        else: 
            Map_left.append(user_sim_lists[i])

    # print('Left:', Map_left, '\n', '\n', 'Right:', Map_right)
    
    Mapping_list = []

    for i in range(len(Map_left)):
        Mapping_list.append(Map_left[i][0])
       
    Mapping_list.append(User_id)
        
    for i in range(len(Map_right)):
        Mapping_list.append(Map_right[len(Map_right)-1-i][0])

    # Mapping_list
    
    def maprange( a, b, s):
        (a1, a2), (b1, b2) = a, b
        return  b1 + ((s - a1) * (b2 - b1) / (a2 - a1))

    Mapped_list = []

    for s in range(360):
        Mapped_list.append(int(maprange( (0, 360), (0, len(Mapping_list)), s)))

    # len(Mapped_list)
    # Mapped_list
    
    Final_mapped = []

    for i in range(len(Mapped_list)):
        Final_mapped.append(Mapping_list[Mapped_list[i]])
    
    return Final_mapped

In [32]:
sorted_list_politics = []

for i in range(Number_of_users):
    sorted_list_politics.append(sorting(m_dict_politics, i))
    
    
sorted_list_climate = []

for i in range(Number_of_users):
    sorted_list_climate.append(sorting(m_dict_climate, i))
    
    
sorted_list_technology = []

for i in range(Number_of_users):
    sorted_list_technology.append(sorting(m_dict_technology, i))
    
    
sorted_list_culture = []

for i in range(Number_of_users):
    sorted_list_culture.append(sorting(m_dict_culture, i))
    
    
sorted_list_health = []

for i in range(Number_of_users):
    sorted_list_health.append(sorting(m_dict_health, i))
    
    
sorted_list_business = []

for i in range(Number_of_users):
    sorted_list_business.append(sorting(m_dict_business, i))

# Connection to interface via OOCSI

In [35]:
# Copyright (c) 2017 Mathias Funk
# This software is released under the MIT License.
# http://opensource.org/licenses/mit-license.php

from oocsi import OOCSI
import time
from random import random
import math
import random


check_list1 = [3.9741147067910885]
check_list2 = [4]
check_list3 = [7]
check_list4 = ['business']
check_list5 = [45,45,45,45]
state = 1

# User_sim_list1 = []
# User_sim_list2 = []
# User_sim_list3 = []
# User_sim_list4 = []
# User_sim_list5 = []

def receiveEvent(sender, recipient, event):
    check_list1[0] = event.get('rad_b')
    check_list2.append(event.get('rad_s'))
    check_list3.append(event.get('user'))
    check_list4.append(event.get('topic'))
    check_list5.append(event.get('numb_profiles'))

## Change the number everytime you run the code
o = OOCSI("Jim_FBP_algorithm190", "oocsi.id.tue.nl")

o.subscribe('popping_bubbles_interface', receiveEvent)

while 1:   
    if state == 1:
        User_id = int(check_list3[(len(check_list3)-1)])
#         print(User_id)
        Select_numb_users = check_list5[(len(check_list5)-1)]
        
        User_sim_list_politics = sorted_list_politics[User_id]
        User_sim_list_business = sorted_list_business[User_id]
        User_sim_list_technology = sorted_list_technology[User_id]
        User_sim_list_culture = sorted_list_culture[User_id]
        User_sim_list_health = sorted_list_health[User_id]
        User_sim_list_climate = sorted_list_climate[User_id]
        
        if len(User_sim_list_politics) == 0:
            print('empty')
        
        Select_User_sim_list_politics = User_sim_list_politics[(len(User_sim_list_politics)-Select_numb_users):(len(User_sim_list_politics)-1)]
        Select_User_sim_list_business = User_sim_list_business[(len(User_sim_list_business)-Select_numb_users):(len(User_sim_list_business)-1)]
        Select_User_sim_list_technology = User_sim_list_technology[(len(User_sim_list_technology)-Select_numb_users):(len(User_sim_list_technology)-1)]
        Select_User_sim_list_culture = User_sim_list_culture[(len(User_sim_list_culture)-Select_numb_users):(len(User_sim_list_culture)-1)]
        Select_User_sim_list_health = User_sim_list_health[(len(User_sim_list_health)-Select_numb_users):(len(User_sim_list_health)-1)]
        Select_User_sim_list_climate = User_sim_list_climate[(len(User_sim_list_climate)-Select_numb_users):(len(User_sim_list_climate)-1)]
        
        F_m_politics = mapp(Select_User_sim_list_politics)
        F_m_business = mapp(Select_User_sim_list_business)
        F_m_technology = mapp(Select_User_sim_list_technology)
        F_m_culture = mapp(Select_User_sim_list_culture)
        F_m_health = mapp(Select_User_sim_list_health)
        F_m_climate = mapp(Select_User_sim_list_climate)

        if (check_list4[(len(check_list4)-1)] == 'politics'):
            Final_mapped = F_m_politics
        elif (check_list4[(len(check_list4)-1)] == 'business'):
            Final_mapped = F_m_business
        elif (check_list4[(len(check_list4)-1)] == 'tech'):
            Final_mapped = F_m_technology
        elif (check_list4[(len(check_list4)-1)] == 'culture'):
            Final_mapped = F_m_culture
        elif (check_list4[(len(check_list4)-1)] == 'health'):
            Final_mapped = F_m_health
        elif (check_list4[(len(check_list4)-1)] == 'climate'):
            Final_mapped = F_m_climate
            
        current_profile = Final_mapped[int(abs(math.degrees(check_list1[len(check_list1)-1])))]
        n_p = 'user'+ str(current_profile)    
        n_p_2 = globals()[n_p].loc[globals()[n_p]['topic'] == check_list4[(len(check_list4)-1)]]
        n_p_3 = n_p_2.reset_index()
        fav = n_p_3['favorite_count']
        
#         print(check_list5)
        print(Select_numb_users)
        print(F_m_climate)
        print(Final_mapped)
        
        if (len(n_p_3) == 0):
            state = 2
        else:
            state = 3
    
    elif state == 2:
        current_profile = Final_mapped[int(abs(math.degrees(check_list1[len(check_list1)-1]))) + 1]
        print(state)
        state = 1
        
    elif state == 3:
#         nn = random.randint(0,(len(n_p_3)-1))
#         print(current_profile)

        message = {}
        message['start'] = int(333)
        message['num_Users'] = Final_mapped[int(abs(math.degrees(check_list1[len(check_list1)-1])))]
        message['text'] = n_p_3.iloc[fav.idxmax()]['full_text']
        message['current_profile'] = n_p
        
        o.send('popping_bubbles_algorithm', message)
        print(n_p)

        state = 1
   
    # wait and continue
    time.sleep(1)

[Jim_FBP_algorithm190]: connecting to oocsi.id.tue.nl port 4444
[Jim_FBP_algorithm190]: connection established
[Jim_FBP_algorithm190]: subscribed to popping_bubbles_interface
45
[40, 40, 40, 40, 40, 40, 40, 40, 34, 34, 34, 34, 34, 34, 34, 34, 29, 29, 29, 29, 29, 29, 29, 29, 31, 31, 31, 31, 31, 31, 31, 31, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 30, 30, 30, 30, 30, 30, 30, 30, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 16, 16, 16, 16, 16, 16, 16, 16, 32, 32, 32, 32, 32, 32, 32, 32, 44, 44, 44, 44, 44, 44, 44, 44, 42, 42, 42, 42, 42, 42, 42, 42, 13, 13, 13, 13, 13, 13, 13, 13, 46, 46, 46, 46, 46, 46, 46, 46, 12, 12, 12, 12, 12, 12, 12, 12, 45, 45, 45, 45, 45, 45, 45, 45, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 9, 9, 9, 9, 9, 9, 9, 9, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 

user22
45
[40, 40, 40, 40, 40, 40, 40, 40, 34, 34, 34, 34, 34, 34, 34, 34, 29, 29, 29, 29, 29, 29, 29, 29, 31, 31, 31, 31, 31, 31, 31, 31, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 30, 30, 30, 30, 30, 30, 30, 30, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 16, 16, 16, 16, 16, 16, 16, 16, 32, 32, 32, 32, 32, 32, 32, 32, 44, 44, 44, 44, 44, 44, 44, 44, 42, 42, 42, 42, 42, 42, 42, 42, 13, 13, 13, 13, 13, 13, 13, 13, 46, 46, 46, 46, 46, 46, 46, 46, 12, 12, 12, 12, 12, 12, 12, 12, 45, 45, 45, 45, 45, 45, 45, 45, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 9, 9, 9, 9, 9, 9, 9, 9, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 26, 26, 26, 26, 26, 26, 26, 26, 23, 23, 23, 23, 23, 23, 23, 23, 49, 49, 49, 49, 49, 49, 49, 49, 48, 48, 48, 48, 48, 48, 48, 48, 47, 47, 47, 47, 47, 47, 47, 47, 43, 4

user3
5
[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

user49
5
[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10

user49


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'